In [6]:
# 1) Kurulum (Python hücresi)
import os, sys
os.environ["WANDB_DISABLED"] = "true"

# YOLOv5 repo yoksa indir
%cd /kaggle/working
if not os.path.isdir("yolov5"):
    !git clone --depth 1 https://github.com/ultralytics/yolov5

%cd /kaggle/working/yolov5
!pip install -r requirements.txt -q

/kaggle/working
Cloning into 'yolov5'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 166 (delta 46), reused 104 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (166/166), 992.95 KiB | 9.11 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00a 0:00:01


In [7]:
# 2) Kaggle input'taki datasetini bul ve data.yaml yolunu seç
import os
from pathlib import Path

print("INPUT klasörleri:")
print(os.listdir("/kaggle/input"))

# Buraya kendi dataset klasör adını yaz:
DATASET_DIR = "/kaggle/input/oversampled-fpds"

# data.yaml'in tam yolunu yaz (bazı datasetlerde 'data.yaml', bazılarında 'dataset.yaml' olur)
DATA_YAML = str(next(Path(DATASET_DIR).rglob("*.yaml")))  # ilk yaml'i bulur

print("Seçilen YAML:", DATA_YAML)
assert Path(DATA_YAML).exists(), "YAML bulunamadı."

INPUT klasörleri:
['oversampled-fpds']
Seçilen YAML: /kaggle/input/oversampled-fpds/data.yaml


In [8]:
from pathlib import Path
import yaml

dataset_root = Path("/kaggle/input/oversampled-fpds")  # senin dataset klasörün
src = dataset_root / "data.yaml"
dst = Path("/kaggle/working/data_abs.yaml")

d = yaml.safe_load(src.read_text())

d["train"] = str(dataset_root / "train" / "images")
d["val"]   = str(dataset_root / "valid" / "images")
d["test"]  = str(dataset_root / "test" / "images")

dst.write_text(yaml.safe_dump(d, sort_keys=False))
print("Yazıldı:", dst)
print(dst.read_text())


Yazıldı: /kaggle/working/data_abs.yaml
train: /kaggle/input/oversampled-fpds/train/images
val: /kaggle/input/oversampled-fpds/valid/images
test: /kaggle/input/oversampled-fpds/test/images
nc: 3
names:
- fallen
- sitting
- standing
roboflow:
  workspace: rescue-robot-datasets
  project: oversampled-v1-fpds-mynsa
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/rescue-robot-datasets/oversampled-v1-fpds-mynsa/dataset/1



In [9]:
from pathlib import Path

for p in [
    "/kaggle/input/oversampled-fpds/train/images",
    "/kaggle/input/oversampled-fpds/valid/images",
    "/kaggle/input/oversampled-fpds/test/images",
]:
    print(p, "->", Path(p).exists())


/kaggle/input/oversampled-fpds/train/images -> True
/kaggle/input/oversampled-fpds/valid/images -> True
/kaggle/input/oversampled-fpds/test/images -> True


In [10]:
from pathlib import Path
import re

p = Path("/kaggle/working/yolov5/utils/metrics.py")
t = p.read_text(encoding="utf-8")

m = re.search(r"def bbox_iou\([\s\S]*?\n(?=def |\Z)", t)
print("bbox_iou bulundu mu?:", bool(m))
print(t[m.start():m.start()+2500] if m else "bbox_iou bulunamadı")


bbox_iou bulundu mu?: True
def bbox_iou(box1, box2, xywh=True, GIoU=False, DIoU=False, CIoU=False, eps=1e-7):
    """Calculates IoU, GIoU, DIoU, or CIoU between two boxes, supporting xywh/xyxy formats.

    Input shapes are box1(1,4) to box2(n,4).
    """
    # Get the coordinates of bounding boxes
    if xywh:  # transform from xywh to xyxy
        (x1, y1, w1, h1), (x2, y2, w2, h2) = box1.chunk(4, -1), box2.chunk(4, -1)
        w1_, h1_, w2_, h2_ = w1 / 2, h1 / 2, w2 / 2, h2 / 2
        b1_x1, b1_x2, b1_y1, b1_y2 = x1 - w1_, x1 + w1_, y1 - h1_, y1 + h1_
        b2_x1, b2_x2, b2_y1, b2_y2 = x2 - w2_, x2 + w2_, y2 - h2_, y2 + h2_
    else:  # x1, y1, x2, y2 = box1
        b1_x1, b1_y1, b1_x2, b1_y2 = box1.chunk(4, -1)
        b2_x1, b2_y1, b2_x2, b2_y2 = box2.chunk(4, -1)
        w1, h1 = b1_x2 - b1_x1, (b1_y2 - b1_y1).clamp(eps)
        w2, h2 = b2_x2 - b2_x1, (b2_y2 - b2_y1).clamp(eps)

    # Intersection area
    inter = (b1_x2.minimum(b2_x2) - b1_x1.maximum(b2_x1)).clamp(0) * (
   

In [11]:
import subprocess, textwrap, os, sys, re
from pathlib import Path

t = Path("/kaggle/working/yolov5/utils/metrics.py").read_text(encoding="utf-8")
for kw in ["DIoU", "CIoU", "GIoU", "rho2", "c2", "cw", "ch", "v ="]:
    if kw in t:
        print("VAR:", kw)
    else:
        print("YOK:", kw)

VAR: DIoU
VAR: CIoU
VAR: GIoU
VAR: rho2
VAR: c2
VAR: cw
VAR: ch
VAR: v =


In [12]:
from pathlib import Path
import re

root = Path("/kaggle/working/yolov5")
metrics_path = root / "utils" / "metrics.py"
loss_path    = root / "utils" / "loss.py"

metrics = metrics_path.read_text(encoding="utf-8")
loss    = loss_path.read_text(encoding="utf-8")

# 1) bbox_iou imzasına EIoU parametresi ekle
if "EIoU=False" not in metrics:
    metrics_new = re.sub(
        r"def bbox_iou\(([^)]*?)CIoU=False,\s*eps=1e-7\):",
        r"def bbox_iou(\1CIoU=False, EIoU=False, eps=1e-7):",
        metrics
    )
    if metrics_new == metrics:
        raise RuntimeError("bbox_iou imzası beklenen formatta bulunamadı.")
    metrics = metrics_new

# 2) EIoU bloğunu CIoU return'ünden hemen önce ekle
if "Efficient IoU (EIoU)" not in metrics:
    # 'if CIoU:' satırını bul (senin dosyanda var)
    m = re.search(r"\n\s*if CIoU:\s*#.*\n", metrics)
    if not m:
        # yorum farklıysa fallback
        m = re.search(r"\n\s*if CIoU:\s*\n", metrics)
    if not m:
        raise RuntimeError("'if CIoU:' bloğu bulunamadı.")

    eiou_block = """
            if EIoU:
                # Efficient IoU (EIoU): IoU - (center_dist/c^2 + (dw^2/cw^2) + (dh^2/ch^2))
                cw2 = cw**2 + eps
                ch2 = ch**2 + eps
                return iou - (rho2 / c2 + (w2 - w1).pow(2) / cw2 + (h2 - h1).pow(2) / ch2)
"""

    # EIoU bloğunu 'if CIoU:' satırından hemen önce ekle
    insert_pos = m.start()
    metrics = metrics[:insert_pos] + eiou_block + metrics[insert_pos:]

# 3) loss.py: bbox_iou(..., CIoU=True) -> bbox_iou(..., EIoU=True)
# YOLOv5 loss.py'de genelde "CIoU=True" kullanılır
if "EIoU=True" not in loss:
    loss_new = re.sub(
        r"bbox_iou\(([^)]*?),\s*CIoU=True\)",
        r"bbox_iou(\1, CIoU=False, EIoU=True)",
        loss
    )
    if loss_new == loss:
        raise RuntimeError("loss.py içinde 'bbox_iou(..., CIoU=True)' kalıbı bulunamadı.")
    loss = loss_new

metrics_path.write_text(metrics, encoding="utf-8")
loss_path.write_text(loss, encoding="utf-8")

print("OK — EIoU patch uygulandı (metrics.py + loss.py).")

OK — EIoU patch uygulandı (metrics.py + loss.py).


In [13]:
!grep -n "EIoU" /kaggle/working/yolov5/utils/metrics.py | head -n 20
!grep -n "EIoU=True" /kaggle/working/yolov5/utils/loss.py | head -n 20


238:def bbox_iou(box1, box2, xywh=True, GIoU=False, DIoU=False, CIoU=False, EIoU=False, eps=1e-7):
271:            if EIoU:
272:                # Efficient IoU (EIoU): IoU - (center_dist/c^2 + (dw^2/cw^2) + (dh^2/ch^2))
161:                iou = bbox_iou(pbox, tbox[i], CIoU=False, EIoU=True).squeeze()  # iou(prediction, target)


In [14]:
DATA_YAML = "/kaggle/working/data_abs.yaml"

!python train.py \
  --img 640 \
  --batch 32 \
  --epochs 100 \
  --data "{DATA_YAML}" \
  --weights yolov5n.pt \
  --name yolov5n_eiou \
  --project /kaggle/working/runs \
  --cache

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2026-02-09 20:09:39.967163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770667780.195063     142 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770667780.256247     142 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh